In [2]:
# ! gsutil -m cp gs://noobs-ml/sem-search/with_title_and_abstract_sentences_embeddings.csv .
# ! gsutil -m cp gs://noobs-ml/sem-search/abstract_sentence_embeddings.npy .
# ! gsutil -m cp gs://noobs-ml/sem-search/sentence_alignment.pickle .
# ! gsutil -m cp separate_embeddings.npy gs://noobs-ml/sem-search/
# ! gsutil -m cp retrofitted_separate_sbert_embeddings.npy gs://noobs-ml/sem-search/
# ! gsutil -m cp retrofitted_merged_sbert_embeddings.npy gs://noobs-ml/sem-search/
# ! gsutil -m cp data_all_sbert_embeddings_retro.pkl gs://noobs-ml/sem-search/
! gsutil -m cp gs://noobs-ml/sem-search/top_100_authors.csv .

Copying gs://noobs-ml/sem-search/top_100_authors.csv...
/ [1/1 files][  3.9 MiB/  3.9 MiB] 100% Done                                    
Operation completed over 1 objects/3.9 MiB.                                      


In [2]:
import pandas as pd
import ast
import numpy as np
import pickle
from copy import deepcopy
import math
from time import time

In [283]:
data = pd.read_csv("with_title_and_abstract_sentences_embeddings.csv", usecols=["id", "title", "references"])

In [9]:
# ae = data.abstract_sentence_embeddings[0]

In [57]:
# aes = " ".join(ae.replace("\n", "").replace("array([ ", "[").replace('dtype', '').split())
# aes = aes.replace(", =float32)]", "]")


In [87]:
# aess = " ".join(ae.replace("\n", "").replace("      dtype=float32),", "").replace("array(", "").replace(",      dtype=float32)", "").split())

In [90]:
# s = '[0 1 2 3]'
# a = np.fromstring(s[1:-1], dtype=np.int, sep=' ')
# print(a) 

In [88]:
# np.fromstring(aes[2:-2], dtype=np.float32, sep=", ")

In [89]:
# aes[2:-2]

In [98]:
# def create_literal_arrays(array_repr):
#     corrected = " ".join(ae.replace("\n", "").replace("      dtype=float32),", "").replace("array(", "").replace(",      dtype=float32)", "").split())
#     return ast.literal_eval(corrected)

In [ ]:
# data["abstract_sentence_embeddings"] = data.abstract_sentence_embeddings.apply(lambda x: create_literal_arrays(x))

In [ ]:
# 2

In [8]:
# data.drop(["Unnamed: 0", "abstract_sentence_embeddings"], inplace=True, axis=1)

In [9]:
# abstract_sentence_embeddings = np.load("abstract_sentence_embeddings.npy")

In [12]:
# with open('sentence_alignment.pickle', 'rb') as handle:
#     alignment = pickle.load(handle)

In [4]:
# reconstructed_batches = []
# for k, v in alignment.items():
#     start = v["start"]
#     end = v["end"]
#     one_batch = []
#     for i in range(start, end):
#         one_batch.append(abstract_sentence_embeddings[i])
#     reconstructed_batches.append(one_batch)

In [5]:
# len(reconstructed_batches)

In [6]:
# reconstructed_batches_list = [[a.tolist() for a in sublist] for sublist in reconstructed_batches]

In [7]:
batches = np.load("reconstructed_batches_list.npy", allow_pickle=True)

In [10]:
def calculate_average_abstract_embedding(emb_list):
    return np.mean(emb_list, axis=0).tolist()

In [11]:
# all_average_embeddings = [calculate_average_abstract_embedding(el) for el in batches]

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [14]:
# np.save("all_average_embeddings.npy", all_average_embeddings)

In [4]:
# average_embeddings = np.load("all_average_embeddings.npy", allow_pickle=True)

In [3]:
# title_embeddings = np.load("title_embeddings.npy")

In [9]:
len(title_embeddings)

300505

In [17]:
def calculate_average_embedding_abstract_and_title(abstract, title, strategy="merge", model=embedder):

    title_embedding = model.encode(title)
    
    if strategy == "merge":
        abstract_embeddings = model.encode(abstract)
        merged = np.concatenate((title_embedding, abstract_embeddings), axis=0)
        average = np.mean(merged, axis=0)
    elif strategy == "separate":
        average_abstract = get_average_sentences_embedding(abstract)
        average = np.mean([average_abstract, title_embedding[0]], axis=0)
    else:
        print("Warning: wrong strategy is used. Use either 'merge' or 'separate'. Proceeding using the 'merge' strategy.")
        abstract_embeddings = model.encode(abstract)
        merged = np.concatenate((title_embedding, abstract_embeddings), axis=0)
        average = np.mean(merged, axis=0)
        
    return average

In [2]:
# merged_embeddings = []
# for title, batch in zip(title_embeddings, batches):
#     merged = batch.append(title.tolist())
#     mean = np.mean(batch, axis=0)
#     merged_embeddings.append(mean)

In [24]:
merged_embeddings = np.load("merged_embeddings.npy")

In [2]:
np.concatenate((np.array([1,2,3]), np.array([5,6,7])))

array([1, 2, 3, 5, 6, 7])

In [10]:
separate_embeddings = []
for title, avg in zip(title_embeddings, average_embeddings):
    average = np.mean([avg, title], axis=0).tolist()
    separate_embeddings.append(average)

In [14]:
# np.save("separate_embeddings.npy", separate_embeddings)

In [290]:
''' Retrofit word vectors to a lexicon '''
def retrofit(wordVecs, lexicon, numIters):
    newWordVecs = deepcopy(corpus)
    wvVocab = set(newWordVecs.keys())
    loopVocab = wvVocab.intersection(set(lexicon.keys()))
    for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
        for word in loopVocab:
            if word in {2107594154}:
                wordNeighbours = set(lexicon[word]).intersection(wvVocab)
                numNeighbours = len(wordNeighbours)
                #no neighbours, pass - use data estimate
                if numNeighbours == 0:
                    continue
                # the weight of the data estimate is the number of neighbours
                newVec = numNeighbours * wordVecs[word]
                # loop over neighbours and add to new vector (currently with weight 1)
                for ppWord in wordNeighbours:
                    newVec += newWordVecs[ppWord]
                newWordVecs[word] = newVec/(2*numNeighbours)
            else:
                wordNeighbours = set(lexicon[word]).intersection(wvVocab)
                numNeighbours = len(wordNeighbours)
                #no neighbours, pass - use data estimate
                if numNeighbours == 0:
                    continue
                # the weight of the data estimate is the number of neighbours
                newVec = numNeighbours * wordVecs[word]
                # loop over neighbours and add to new vector (currently with weight 1)
                for ppWord in wordNeighbours:
                    newVec += newWordVecs[ppWord]
                newWordVecs[word] = newVec/(2*numNeighbours)
        print("END OF ITERATION")
    return newWordVecs

In [239]:
empty_array = np.array([])

In [247]:
faulty_ids = []
for k, v in corpus.items():
    if np.isnan(v[0]):
        faulty_ids.append(k)

In [287]:
data_no_faulty_ids = data.query("id not in @faulty_ids")

In [289]:
data_no_faulty_ids.head()

,title,id,references,merged_sbert_embeddings,separate_sbert_embeddings
0,Texture processing of extruded YBa2Cu3O6+x wir...,2046483935,"['5326298', '1514024641', '1976557467', '19784...","[0.4932159246876836, 0.17480173707008362, -0.4...","[0.46044804171348613, 0.22540580729643503, -0...."
1,MetaBar - a tool for consistent contextual dat...,2046485931,"['1878193911', '1966380925', '1969538518', '19...","[0.08118600398302078, -0.6622746661305428, -0....","[0.22415918608506522, -0.8397057900826136, -0...."
2,Erroneous efficiency reports harm organic sola...,2046488594,"['1976071035', '1976716245', '1982320157', '19...","[-0.42702873051166534, 0.25434818118810654, -0...","[-0.42702873051166534, 0.25434818118810654, -0..."
3,Cytokines Score a Knockout Harnessing Gene Tar...,2046488964,"['47956823', '1602113709', '2002209097', '2006...","[0.3700085314026532, -0.12047294867583193, 0.0...","[0.31110948682180606, 0.12373845684730138, 0.0..."
4,Hemodynamic Effects of Diuresis at Rest and Du...,2046489009,"['138127152', '1615932296', '1968828189', '197...","[-0.1697837064663569, 0.2592622209340334, 0.50...","[-0.3549113192341544, 0.34182563254779036, 0.4..."


In [252]:
# data[data['id'].isin(faulty_ids)]

In [273]:
# for ae in average_embeddings:
#     try:
#         if None in ae:
#             print(ae)
#     except:
#         print(ae)

In [231]:
# for i in range(8):
#     for i in range(2):
#         print(newCorpus[2096525273][:5])

In [284]:
# data["merged_sbert_embeddings"] = merged_embeddings

In [285]:
# data["separate_sbert_embeddings"] = separate_embeddings

In [126]:
# merged_embeddings = [x.tolist() for x in merged_embeddings]

In [291]:
def normalize_embedding(embedding):
    embedding = np.array(embedding)
    return embedding / math.sqrt((embedding**2).sum() + 1e-6)

In [334]:
def build_lexicon_and_corpus(data):
    lexicon = {}
    corpus = {}
    for pi, pref, pemb in zip(data.id, data.references, data.merged_sbert_embeddings):
        lexicon[pi] = [int(i) for i in ast.literal_eval(pref)]
        corpus[pi] = normalize_embedding(pemb)
    return lexicon, corpus

In [335]:
lexicon, corpus = build_lexicon_and_corpus(data_no_faulty_ids)

In [336]:
start = time()
retrofitted_merged = retrofit(corpus, lexicon, 10)
time() - start

END OF ITERATION
END OF ITERATION
END OF ITERATION
END OF ITERATION
END OF ITERATION
END OF ITERATION
END OF ITERATION
END OF ITERATION
END OF ITERATION
END OF ITERATION


108.88173651695251

In [316]:
# data_no_faulty_ids['references'] = data_no_faulty_ids['references'].apply(lambda d: d if d != "None-placeholder" else '[]')

In [305]:
corpus[2046483935][:5]

array([ 0.03045467,  0.01490865, -0.02679888,  0.03669132,  0.04959443])

In [306]:
retrofitted[2046483935][:5]

array([ 0.02864495, -0.00012356, -0.01523238,  0.06767576,  0.03402121])

In [342]:
retro_list = [na.tolist() for na in retrofitted_merged.values()]

In [326]:
# data_no_faulty_ids.reset_index(drop=True, inplace=True)

In [343]:
np.save("retrofitted_merged_sbert_embeddings.npy", retro_list)

In [1]:
# np.load("retrofitted_merged_sbert_embeddings.npy", allow_pickle=True)

In [344]:
data_no_faulty_ids["retrofitted_merged_bert_embeddings"] = retro_list

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [348]:
data_no_faulty_ids.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300476 entries, 0 to 300475
Data columns (total 7 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   title                                 300476 non-null  object
 1   id                                    300476 non-null  int64 
 2   references                            300476 non-null  object
 3   merged_sbert_embeddings               300476 non-null  object
 4   separate_sbert_embeddings             300476 non-null  object
 5   retrofitted_separate_bert_embeddings  300476 non-null  object
 6   retrofitted_merged_bert_embeddings    300476 non-null  object
dtypes: int64(1), object(6)
memory usage: 7.1 GB


In [349]:
# data_no_faulty_ids.to_pickle("data_all_sbert_embeddings_retro.pkl")

In [3]:
data = pd.read_pickle("data_all_sbert_embeddings_retro.pkl")

In [4]:
data

,title,id,references,merged_sbert_embeddings,separate_sbert_embeddings,retrofitted_separate_bert_embeddings,retrofitted_merged_bert_embeddings
0,Texture processing of extruded YBa2Cu3O6+x wir...,2046483935,"['5326298', '1514024641', '1976557467', '19784...","[0.4932159246876836, 0.17480173707008362, -0.4...","[0.46044804171348613, 0.22540580729643503, -0....","[0.02864494941749328, -0.0001235631464315181, ...","[0.03394347941226632, -0.0033981509988428873, ..."
1,MetaBar - a tool for consistent contextual dat...,2046485931,"['1878193911', '1966380925', '1969538518', '19...","[0.08118600398302078, -0.6622746661305428, -0....","[0.22415918608506522, -0.8397057900826136, -0....","[0.02131571279349928, -0.04861194215507768, -0...","[0.015715423519276094, -0.045007013150989386, ..."
2,Erroneous efficiency reports harm organic sola...,2046488594,"['1976071035', '1976716245', '1982320157', '19...","[-0.42702873051166534, 0.25434818118810654, -0...","[-0.42702873051166534, 0.25434818118810654, -0...","[-0.006884977758239316, -0.0001613655922653738...","[-0.006001515931925572, 0.00134417690433098, -..."
3,Cytokines Score a Knockout Harnessing Gene Tar...,2046488964,"['47956823', '1602113709', '2002209097', '2006...","[0.3700085314026532, -0.12047294867583193, 0.0...","[0.31110948682180606, 0.12373845684730138, 0.0...","[0.020333075987282605, -0.0019313877330480005,...","[0.023164855468410347, -0.012070801538104869, ..."
4,Hemodynamic Effects of Diuresis at Rest and Du...,2046489009,"['138127152', '1615932296', '1968828189', '197...","[-0.1697837064663569, 0.2592622209340334, 0.50...","[-0.3549113192341544, 0.34182563254779036, 0.4...","[-0.01309363138178083, 0.006973425965482617, 0...","[-0.0046112818600819085, 0.003386396169996765,..."
...,...,...,...,...,...,...,...
300471,Effect of leukocyte antibodies and HLA matchin...,263475322,"['1607709818', '1979719355', '2031909680', '20...","[0.22847693581134082, -0.3926648167893291, 0.1...","[0.18440959965320008, -0.5193975425668453, 0.2...","[0.013690730684530597, -0.03856052985778661, 0...","[0.019232013276423963, -0.03305250459027785, 0..."
300472,Monitoring the fermentation process of black t...,263531807,"['29179573', '1242867596', '1425730151', '1964...","[0.16274074547820622, -0.2287636717988385, 0.0...","[0.013922079466283321, -0.310257799224928, 0.2...","[-0.003433796225601209, -0.013865532388897123,...","[0.003758565373991596, -0.011532964138458668, ..."
300473,Obesity in pregnancy: risks and outcome.,263532283,[],"[0.19157937914133072, -0.7784594297409058, 0.2...","[0.19157937914133072, -0.7784594297409058, 0.2...","[0.013164317473479905, -0.05349159768272034, 0...","[0.013164317473479905, -0.05349159768272034, 0..."
300474,Imputing Missing Data for Gene Expression Arrays,263545290,"['1279126693', '2096863518']","[0.1165076924694909, -0.7019703197810385, -0.1...","[0.23742571100592613, -0.7831216868944466, -0....","[0.019550520149182796, -0.049277177423291754, ...","[0.015412138760603175, -0.05006434374774354, -..."
